# Data

### Virus data - daily covid-19 counts by state
Data is downloaded from [Kaggle](https://www.kaggle.com/sudalairajkumar/covid19-in-usa) via API and updated daily with counts from the previous day. Includes metrics like the number of positive, negative, and total cases, as well as number of hospitalized and deaths.

#### Kaggle API 
1. Set up a key by [following these instructions](https://adityashrm21.github.io/Setting-Up-Kaggle/)
2. Store the key file in `~/.kaggle/kaggle.json`

### Census data
Census info is from [American Community Survey 5-Year Data (2009-2018)](https://www.census.gov/data/developers/data-sets/acs-5year.html). 

The "Detailed" datasets are used from 2018 survey results. Only population count estimate variables (variables ending in "E") are used.

Included features by state:
* median age by gender 
* race 
* commute modes
* health insurance

For simplicity, this notebook uses the [census](https://pypi.org/project/CensusData/) python package to extract Census data instead of pulling data from the Census website directly. But that package still requires that you set up an API key with the Census.

#### Census API
1. Get a key from https://api.census.gov/data/key_signup.html and save it in a json file somewhere on your computer. The content of your `census_gov_api_key.json` file should look something like 

```
{"key": "your-api-key-goes-here"}
```

2. Modify the `CENSUS_API_KEY_PATH` variable where it is set below to save to the location you saved your census key to.


Read [this documentation](https://www.census.gov/mso/www/training/pdf/acs_census_api.pdf) if you're interested in learning more about the Census API.

# Metrics
Daily metrics for each state are adjusted by the Census-estimated population in the state and the number of days that have elapsed since the state's first reported positive case.

For example, for the number of positive cases in a particular state we have

$ \textrm{Rate of Positive Cases per 100,000}= \frac{\sum{\textrm{positive cases}}}{\textrm{population estimate (Census)}} \times 100000 $

and

$ \textrm{Adjusted Rate of Positive Cases per 100,000} 
= \textrm{Positive rate per 100,000}\times\frac{\textrm{Days since first positive case}}{1+\textrm{Days since first positive case}} 
$.

The final adjusted rate column uses an adjustment multiplier that gives more confidence to the rate the longer data has been reported in the state. This makes comparisons between states at a given point in time a bit easier.

In [187]:
import json
import requests

import pandas as pd
from pandasql import sqldf
import kaggle
import us

In [199]:
# PARAMETERS THAT YOU MUST CHANGE
CENSUS_API_KEY_PATH = '../../secrets/census_gov_api_key.json' # modify this to the location of your census key

In [188]:
def get_virus_data(needing_refresh=True):
    """Pulls the latest data from Kaggle and returns a pandas DataFrame
    
    Instructions on how to set up a Kaggle API key:
    https://adityashrm21.github.io/Setting-Up-Kaggle/
    """
    if needing_refresh:
        kaggle.api.dataset_download_files(
            'sudalairajkumar/covid19-in-usa', 
            path='raw', 
            force=True, 
            quiet=True, 
            unzip=True)
    data = pd.read_csv('raw/us_states_covid19_daily.csv')
    data['obs_dt'] = pd.to_datetime(data.date, format='%Y%m%d')
    return data

In [189]:
def get_census_data(api_key_file=CENSUS_API_KEY_PATH):
    """Pulls 2018 US Census ACS5 data
    
    Extracts select variables. Full list of variables 
    is at https://api.census.gov/data/2018/acs/acs5/variables.html
    """
    # pre-set list of census variables to extract
    variable_names = {
      'B01003_001E': 'population_total',
      'B01002_002E': 'median_age_male',
      'B01002_003E': 'median_age_female',
      'B02001_002E': 'race_white_alone',
      'B02001_003E': 'race_afr_alone',
      'B02001_005E': 'race_asian_alone',
      'B03003_002E': 'hisp_not_hisp',
      'B08301_003E': 'transport_drive_alone',
      'B08301_010E': 'transport_public',
      'B08301_019E': 'transport_walk',
      'B992701_003E': 'healthins_none'
    }
    # census api key 
    with open(api_key_file) as f:
        census_api = json.load(f)
    census_key = census_api['key']
    
    # object from census python package
    census = Census(census_key)
    variables = tuple(variable_names.keys())
    # state* pulls state-level data for all states
    raw = census.acs5.get((variables), geo={'for': 'state:*'}, year=2018)
    
    # data cleanup
    data = pd.DataFrame(raw)
    data.rename(columns=variable_names, inplace=True)
    data['state_cd'] = [us.states.lookup(fips).abbr for fips in data['state']]
    data['state_name'] = [us.states.lookup(fips).name for fips in data['state']]
    return data

In [190]:
census_df = get_census_data()

In [191]:
virus_df = get_virus_data()

In [192]:
sql="""
  select 
    state,
    min(obs_dt)  as obs_dt
  from virus_df 
  where positive is not null
  group by 1
"""
firstcase = sqldf(sql, globals())

In [193]:
sql = """
    select 
        c.state_name,
        s.state,
        strftime('%Y-%m-%d', s.obs_dt) as obs_dt,
        strftime('%Y-%m-%d', f.obs_dt) as first_obs_dt,
        coalesce(s.positive,0) as positive,
        coalesce(s.negative,0) as negative,
        coalesce(s.pending,0) as pending,
        coalesce(s.hospitalized,0) as hospitalized,
        coalesce(s.death,0) as death,
        coalesce(s.total,0) as total,
        c.population_total as population,
        c.median_age_male, 
        c.median_age_female,
        c.population_total, 
        c.race_white_alone, 
        c.race_afr_alone,
        c.race_asian_alone, 
        c.hisp_not_hisp, 
        c.transport_drive_alone, 
        c.transport_public, 
        c.transport_walk,
        c.healthins_none
    from virus_df as s
    inner join census_df as c
        on s.state = c.state_cd
    left join firstcase as f 
        on s.state = f.state
"""
data = sqldf(sql, globals())

In [194]:
data['obs_dt'] = pd.to_datetime(data.obs_dt, format='%Y-%m-%d')
data['first_obs_dt'] = pd.to_datetime(data.first_obs_dt, format='%Y-%m-%d')

In [195]:
data['time_elapsed_since_firstpos'] = data['obs_dt'] - data['first_obs_dt']
data['days_elapsed_since_firstpos'] = data['time_elapsed_since_firstpos'].astype('timedelta64[D]').astype(int)
data['pos_per100k'] = 100000.0 * (data['positive'] / data['population'])
data['death_per100k'] =  100000.0 * (data['death'] / data['population'])

In [196]:
data['adj_multiplier'] = (data.days_elapsed_since_firstpos / (1.0 + data.days_elapsed_since_firstpos))
data['adj_pos_per100k'] = data.pos_per100k * data.adj_multiplier
data['adj_death_per100k'] = data.death_per100k * data.adj_multiplier

In [197]:
data.to_csv('data/results.csv')